# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [67]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [62]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,73.47,78,100,37.09,PN,Tue Jul 25 21:13:03 2023
1,1,iskateley,67.6803,53.1512,63.32,75,65,3.67,RU,Tue Jul 25 21:15:33 2023
2,2,koidu,8.4167,-10.8333,71.44,100,100,1.07,SL,Tue Jul 25 21:15:33 2023
3,3,grytviken,-54.2811,-36.5092,26.78,83,83,9.71,GS,Tue Jul 25 21:13:06 2023
4,4,rawson,-43.3002,-65.1023,42.46,78,24,11.12,AR,Tue Jul 25 21:13:50 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = 'Humidity',
    color = 'City'
)

# Display the map
humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_cities = city_data_df[(city_data_df['Max Temp'] < 80) &
                            (city_data_df['Max Temp'] > 60) &
                            (city_data_df['Humidity'] < 50) &
                            (city_data_df['Cloudiness'] <= 70)
                        ]

# Drop any rows with null values

ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,ust-nera,64.5667,143.2000,68.79,49,54,4.61,RU,Tue Jul 25 21:15:51 2023
83,83,mokhsogollokh,61.4681,128.9203,77.83,31,11,8.72,RU,Tue Jul 25 21:13:42 2023
87,87,ain el bya,35.8039,-0.3018,71.89,44,0,2.84,DZ,Tue Jul 25 21:15:55 2023
119,119,kosh-agach,50.0000,88.6667,65.30,44,4,2.08,RU,Tue Jul 25 21:16:07 2023
129,129,kaynarca,41.0308,30.3075,79.59,46,0,4.94,TR,Tue Jul 25 21:16:10 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_cities.iloc[:,[1, 8, 2, 3, 4,]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Max Temp,Hotel Name
69,ust-nera,RU,64.5667,143.2000,68.79,
83,mokhsogollokh,RU,61.4681,128.9203,77.83,
87,ain el bya,DZ,35.8039,-0.3018,71.89,
119,kosh-agach,RU,50.0000,88.6667,65.30,
129,kaynarca,TR,41.0308,30.3075,79.59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [64]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
          'filter':'',\
          'bias':'',\
          'apiKey':geoapify_key,\
          'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
ust-nera - nearest hotel: гостиница "Солнечная"
mokhsogollokh - nearest hotel: No hotel found
ain el bya - nearest hotel: فندق أريحا
kosh-agach - nearest hotel: Заря
kaynarca - nearest hotel: No hotel found
khovd - nearest hotel: Khovd hotel
arenapolis - nearest hotel: No hotel found
mikhaylovskoye - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
moose jaw - nearest hotel: Temple Gardens Mineral Spa Resort
talnakh - nearest hotel: Талнах
sao vicente - nearest hotel: Pousada Vitória
ulladulla - nearest hotel: Hotel Marlin
valentin gomez farias - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
altai - nearest hotel: Altai hotel 40000-100000T
portland - nearest hotel: the Hoxton
sebdou - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
mogocha - nearest hotel: 7Ka
qarauyl - nearest hotel: No hotel found
kochenevo - nearest hotel: No hotel found
port augusta 

,City,Country,Lat,Lng,Max Temp,Hotel Name
69,ust-nera,RU,64.5667,143.2000,68.79,"гостиница ""Солнечная"""
83,mokhsogollokh,RU,61.4681,128.9203,77.83,No hotel found
87,ain el bya,DZ,35.8039,-0.3018,71.89,فندق أريحا
119,kosh-agach,RU,50.0000,88.6667,65.30,Заря
129,kaynarca,TR,41.0308,30.3075,79.59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
ideal_city_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True, 
        tiles = 'OSM', 
        frame_width = 700, 
        frame_height = 500, 
        color='City',
        alpha=0.6,
        hover=True,
        hover_cols='all',
        use_index=False,
        scale= 1)
# Display the map
ideal_city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp,Hotel Name)